In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "at010-agent-chat"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='fc5d61ff-b9f5-4898-a5a0-a88433955398', created_at=datetime.datetime(2025, 8, 20, 22, 47, 26, 837621, tzinfo=TzInfo(UTC)), name='e292b96b-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 20, 22, 47, 26, 837621, tzinfo=TzInfo(UTC)))


In [ ]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups. Think hard and think multiple times per step."},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='f3c427de-c8cd-49d5-a062-d7500b9948f7', agent_id='a1abb90e-c673-4448-a4e2-841170568840', sequence_id=1044, task_id='fc5d61ff-b9f5-4898-a5a0-a88433955398', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups. Think multiple times per step.', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 20, 22, 47, 26, 873787, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=60,
)

╭───────────────────────── USER [08/20/2025 22:47:26] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups. Think multiple times │
│ per step.                                                                    │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/20/2025 22:47:34] ─────────────────────────╮
│ 🧠 Reasoning                                                                 │
│                                                                              │
│ Fetching latest AI news                                                      │
│                                                                              │
│ I’m getting ready to fetch the latest news using the web search tool. I      │
│ think I’ll use the functions available and set it up with the model default. │
│ I’ll choose search input focusing on "latest AI news August 2025,"           │
│ especially regarding startups. I’ll consider everything relevant, like       │
│ funding, layoffs, newly launched models, acquisitions, regulations, and      │
│ venture capital. I just need to run the web search now to gather this        │
│ information.                                                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/20/2025 22:47:36] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest AI news August 2025 AI startups funding layoffs          │
│  acquisitions new models regulation venture capital August 2025",            │
│    "model": "gpt-4o-mini",                                                   │
│    "type": "web_search_preview",                                             │
│    "search_context_size": "high"                                             │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [08/20/2025 22:47:47] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "As of August 20, 2025, the artificial intelligence (AI) sector   │
│  has experienced significant developments across funding, acquisitions,      │
│  layoffs, new model releases, and regulatory changes. Here's an overview of  │
│  the latest AI news:\n\n**Funding and Acquisitions:**\n\n- **TinyFish's $47  │
│  Million Series A Funding:** TinyFish, an AI startup specializing in         │
│  automating complex online tasks for enterprises, secured $47 million in a   │
│  Series A funding round led by ICONIQ Capital. The investment aims to enhan  │
│  product development and expand market reach.                                │
│  ([reuters.com](https://www.reuters.com/technology/ai-agent-startup-tinyfis  │
│  raises-47-million-iconiq-led-round-2025-08-20/?utm_source=openai))\n\n-     │
│  **Databricks' Valuation Surge:** Databricks, a San Francisco-based analyti  │
│  company, is poised to surpass a $100 billion valuation in its upcoming      │
│  funding round. This reflects strong investor enthusiasm for AI ventures,    │
│  with plans to advance product development and pursue mergers and            │
│  acquisitions within the AI space.                                           │
│  ([reuters.com](https://www.reuters.com/business/databricks-eyes-over-100-b  │
│  lion-valuation-investors-back-ai-growth-plans-2025-08-19/?utm_source=opena  │
│  )\n\n- **OpenAI's Acquisition of io:** OpenAI announced the $6.5 billion    │
│  acquisition of io, an AI hardware startup founded by former Apple designer  │
│  Jony Ive. The merger aims to integrate hardware and software development,   │
│  with Ive assuming creative responsibilities across OpenAI.                  │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI?utm_source=openai  │
│  \n\n**Layoffs:**\n\n- **Recruit Holdings' Workforce Reduction:** Recruit    │
│  Holdings, the parent company of Indeed and Glassdoor, is laying off         │
│  approximately 1,300 employees, about 6% of its HR technology unit. This mo  │
│  is part of a strategic shift toward integrating AI technologies and         │
│  streamlining operations.                                                    │
│  ([linkedin.com](https://www.linkedin.com/pulse/ai-news-funding-updates-fro  │
│  last-24-hours11th-july-2025-anshuman-jha-2d20c?utm_source=openai))\n\n-     │
│  **Meta's Staff Reduction:** Meta announced a 5% reduction in its workforce  │
│  targeting \"low performers\" as the company prepares for an \"intense       │
│  year.\" This decision reflects ongoing efforts to optimize operations amid  │
│  evolving market conditions.                                                 │
│  ([techcrunch.com](https://techcrunch.com/2025/07/31/tech-layoffs-2025-list  │
│  utm_source=openai))\n\n**New AI Models:**\n\n- **OpenAI's GPT-5 Release:**  │
│  OpenAI released GPT-5, the latest iteration of its language model, offerin  │
│  enhanced capabilities and performance. This release signifies continued     │
│  advancements in AI language processing.                                     │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/2025_in_artificial_intel  │
│  gence?utm_source=openai))\n\n- **Google's A.I. Mode and Veo 3:** Google     │
│  introduced A.I. Mode, a feature on its search engine powered by the Gemini  │
│  model, and Veo 3, a new state

Streaming timed out after 60 seconds - returning collected messages
